데이터 수집 및 전처리
--------------

In [28]:
# 고객의 성향 분석하기
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
# importing dataset
dataset = pd.read_csv('data/Churn_Modelling.csv')

In [10]:
X = dataset.iloc[:, 3:13].values # 중요한 지표들
y = dataset.iloc[:, 13].values # 평가지표

In [11]:
X # 문자열로 저장된 데이터들....

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [12]:
# Encoding Categorical Data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [13]:
X # 정수형태로 저장된 데이터들
# 하지만 binary 형태로 바꿔줘야 한다
# isFrance? isGermany, isSpain의 변수 형태로 바꿔줘야 한다 -> OnehotEncoding

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [24]:
onehotencoder = OneHotEncoder(categorical_features=[1])
X = onehotencoder.fit_transform(X).toarray()

C:\Users\default.DESKTOP-S5Q9GAA\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [34]:
# 하지만 isFrace, isGermany 아니면 자동으로 Spain인까 연산 줄이는 차원에서..
# 첫 번째 열 삭제
# To avoid Dummy variable Trap
X = X[:, 1:]
X.shape

(10000, 11)

In [36]:
# 훈련, 시험 데이터 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

(8000, 11)

In [27]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

모델 구현
------

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [2]:
# ANN의 템플릿 생성
classifier = Sequential()

In [3]:
# Adding the input layer and the first hiddne layer
classifier.add(Dense(units=6,  # hidden layer의 수(=input layer과 output layer의 평균)
                    kernel_initializer='uniform', # W 초기화 분포
                    activation='relu', # 활성화 함수
                    input_dim=11)) # input layer의 수

# Adding the second hidden layer
classifier.add(Dense(units=6,
                    kernel_initializer='uniform',
                    activation='relu'))

# Adding the output layer
classifier.add(Dense(units=1,
                    kernel_initializer='uniform',
                    activation='sigmoid'))

모델 학습 및 성능 평가
-----------

In [5]:
# 지금까지 만들어진 Layer를 ANN으로 묶어서 처리하는 compile 과정이 필요함
# Compling the ANN
classifier.compile(optimizer='adam', # Stochastic Gradient Descent를 최적화 시켜주는 Adam optimizer
                  loss = 'binary_crossentropy', # 손실 함수
                  metrics = ['accuracy']) # 평가 지표 : 정확도

In [37]:
# batch_size : 학습시킬 군집의 단위
# epoch : 전체 단위가 학습되는 횟수
# epoch가 10이라면 batch_size만큼 10회 돌리겠다는 뜻
# Fitting the ANN to the Traning set
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 1s 156us/step - loss: 0.5704 - acc: 0.7847
Epoch 2/100
8000/8000 [==============================] - 1s 111us/step - loss: 0.5255 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 111us/step - loss: 0.5166 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 117us/step - loss: 0.5088 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.5087 - acc: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 108us/step - loss: 0.5047 - acc: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 110us/step - loss: 0.5026 - acc: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.5025 - acc: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 112us/step - loss: 0.5003 - acc: 0.7960
Epoch 10/100
8000/8000 [==============================] - 1s 109us/step - loss: 0.5011 - acc: 0.7960

In [39]:
# Predicting the Test set results
# 잘 맞췄는지 여부를 확인하기 위해서..
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [41]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1595,    0],
       [ 405,    0]], dtype=int64)